In [1]:
import os
import sys
sys.path.append("..")

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import distortion_layers as ly

# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) # ignores warnings

### Create distortion layers

In [2]:
dcs = {
#     1:{
#         "contrast": {"factor": [0.2, 0.2]},
#         "color": {"factor": [5,0,5]},
#         "blur": {"filter_shape": 1, "sigma": 0.},  # kernel size is 'filter_shape * 2 + 1'
#         "brightness": {"factor": 0.3}
#     },
    1:{
        "contrast": {"factor": [0.3, 0.3]},
        "color": {"factor": [5,0,5]},
        "blur": {"filter_shape": 4, "sigma": 0.},  # kernel size is 'filter_shape * 2 + 1'
        "brightness": {"factor": 0.3}
#         "brightness": {"lower":1.2999, "upper":1.3}
    },
    2:{
        "contrast": {"factor": [0.3, 0.3]},
        "color": {"factor": [5,0,5]},
        "blur": {"filter_shape": 2, "sigma": 1.},
        "brightness": {"factor": 0.3}
#         "brightness": {"lower":0.7, "upper":1.3}
    },
    3:{
        "contrast": {"factor": [0.4, 0.6]},
        "color": {"factor": [20,0,20]},
        "blur": {"filter_shape": 2, "sigma": 5.},
        "brightness": {"factor": 0.5}
#         "brightness": {"lower":0.7, "upper":1.3}
    },
    4:{
        "contrast": {"factor": [0.4, 0.6]},
        "color": {"factor": [20,20,20]},
        "blur": {"filter_shape": 4, "sigma": 5.},
        "brightness": {"factor": 0.5}
#         "brightness": {"lower":0.7, "upper":1.3}
    }
}


layers = [
    ly.DistortionLayer(layers=[
        ly.RandomColorByChannel(**dcs[i]["color"]), 
        tf.keras.layers.RandomContrast(**dcs[i]["contrast"]),
#         ly.RandomBrightness(**dcs[i]["brightness"]),
        tf.keras.layers.RandomBrightness(**dcs[i]["brightness"]),
        ly.RandomGaussianBlur(**dcs[i]["blur"])])
    for i in dcs
]


2022-06-27 16:37:02.747987: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-27 16:37:02.754055: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-27 16:37:02.754507: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-27 16:37:02.755147: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

AttributeError: module 'keras.api._v2.keras.layers' has no attribute 'RandomBrightness'

### Load images

#### Option 1: own data set
If the notebook is ran from Docker and an absolute path wants to be provided for the train_path, make sure that the path corresponds to the docker path based on the volume used with docker run.

Otherwise, just load the data into the data folder

In [ ]:
data_path = "../data"
tile_size = 128
batch_size = 20

gen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split = 0.2
)
flow = gen.flow_from_directory(
    directory=data_path,
    target_size=(tile_size,tile_size),
    color_mode='rgb',  # 'rgb' or 'grayscale' 
    batch_size=batch_size,
)


In [ ]:
def visualize_augmentations(num_images):
    # Sample a batch from a dataset
    batch = next(iter(flow))
    if type(batch) == tuple:
        images = batch[0][:num_images]
    else:
        images = batch[:num_images]
    # Apply augmentations
    augmented_images = zip(
        images.astype(int),
        layers[0](images).numpy().astype(int),
        layers[0](images).numpy().astype(int),
        layers[1](images).numpy().astype(int),
        layers[1](images).numpy().astype(int),
        layers[2](images).numpy().astype(int),
        layers[2](images).numpy().astype(int),
        layers[3](images).numpy().astype(int),
        layers[3](images).numpy().astype(int),
    )
    row_titles = [
        "Original:",
        "DC1:",
        "DC1:",
        "DC2:",
        "DC2:",
        "DC3:",
        "DC3:",
        "DC4:",
        "DC4:",
    ]
    plt.figure(figsize=(num_images * 2.2, 9 * 2.2), dpi=100)
    for column, image_row in enumerate(augmented_images):
        for row, image in enumerate(image_row):
            plt.subplot(9, num_images, row * num_images + column + 1)
            plt.imshow(image)
            if column == 0:
                plt.title(row_titles[row], loc="left")
            plt.axis("off")
    plt.tight_layout()


visualize_augmentations(num_images=8)

#### Option 2: try it on cifar